### Importing packages

In [1]:
import numpy as np
import glob, os
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from astropy.io import fits
from astropy import wcs
from photutils.aperture import EllipticalAperture as EAp
from photutils.aperture import RectangularAperture as RAp
from photutils.aperture import RectangularAnnulus as RAn
from photutils import aperture_photometry as apphot
from astropy.stats import sigma_clipped_stats
from astropy.cosmology import FlatLambdaCDM

In [2]:
# %matplotlib notebook
%matplotlib widget

### Cosmological parameter

In [3]:
redshift = 0.3527
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
ldist = cosmo.luminosity_distance(redshift).value    # Mpc
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60.    # kpc/arcsec
print(f"z = {redshift:.4f}")
print(f"D_L = {ldist:.2f} Mpc")
print(f"{kpc_per_arcsec:.3f} kpc/arcsec")

z = 0.3527
D_L = 1873.69 Mpc
4.964 kpc/arcsec


### Loading the Gemini GMOS/IFU FOV

In [4]:
# Gemini GMOS/IFU data cube
diG = "/data/jlee/DATA/Gemini/Programs/GN-2019A-Q-215/redux4_700/"
cubname = "cstxeqxbrgN20190611S0257_3D.fits"

# Reading necessary data
ifu_w, ifu_h = 7.0, 5.0    # arcsec
hdr = fits.getheader(diG+cubname, ext=0)
gra, gdec, gpa = hdr["RA"], hdr["DEC"], hdr["PA"]
print("# GMOS/IFU center")
print(f"R.A. = {gra:.5f} deg")
print(f"Decl. = {gdec:.5f} deg")
print(f"PA = {gpa:.1f} deg")
print(f"IFU width = {ifu_w:.1f} arcsec")
print(f"IFU height = {ifu_h:.1f} arcsec")

# GMOS/IFU center
R.A. = 268.02650 deg
Decl. = 44.66746 deg
PA = 85.0 deg
IFU width = 7.0 arcsec
IFU height = 5.0 arcsec


### Aperture photometry for HST/ACS images

In [5]:
# Basic information
diH = "/data/jlee/DATA/HLA/McPartland+16/MACS1752/JFG2/Phot/"
imglist = ["435_ori.fits", "606_ori.fits", "814_ori.fits"]
filt = ["F435W", "F606W", "F814W"]
Amag = [0.109, 0.075, 0.046]    # mag
fwhm = [0.1, 0.1, 0.1]    # arcsec
pixel_scale = 0.05    # arcsec/pixel
wcs_angle = 0.029572035    # deg from DS9 box region

In [6]:
# WCS XY conversion for HST images
h = fits.getheader(diH+imglist[1], ext=0)
w = wcs.WCS(h)
px, py = w.wcs_world2pix(gra, gdec, 1)
print("# GMOS/IFU center")
print(f"(X, Y) = ({px:.4f}, {py:.4f})")

# GMOS/IFU center
(X, Y) = (197.8369, 180.0619)


In [7]:
# Creating the FOV aperture
def make_aperture(calc_sky=False, w_in=None, w_out=None, h_in=None, h_out=None):
    '''
    calc_sky: sky estimation from aperture annulus (boolean, default:False)
        if calc_sky is True, w_in, w_out, h_in, h_out should be given.
    '''
    ap, width, height = [], [], []
    for i in np.arange(len(filt)):
        w0, h0 = ifu_w/pixel_scale, ifu_h/pixel_scale
        if (ifu_h < 2*fwhm[i]):
            w0 *= (2*fwhm[i] / ifu_h)
            h0 *= (2*fwhm[i] / ifu_h)
        ap.append(RAp((px-1, py-1), w=w0, h=h0, theta=(gpa-wcs_angle)*np.pi/180.))
        width.append(w0)
        height.append(h0)

    if calc_sky:
        an = RAn((px-1, py-1), w_in=w_in, w_out=w_out,
                 h_in=h_in, h_out=h_out, theta=(gpa-wcs_angle)*np.pi/180.)
        return [ap, width, height, an]
    
    else:
        return [ap, width, height]

calc_sky = True
w_in, w_out = ifu_w/pixel_scale+10, ifu_w/pixel_scale+60
h_in, h_out = ifu_h/pixel_scale+10, ifu_h/pixel_scale+60
ap, width, height, an = make_aperture(calc_sky=calc_sky, w_in=w_in, w_out=w_out, h_in=h_in, h_out=h_out)

In [8]:
# Showing the FOV aperture
def plot_aperture(out, directory, imglist, rth, vmin=-0.01, vmax=0.09):
    fig, ax = plt.subplots(1, len(imglist), figsize=(len(imglist)*4, 4))
    for i in np.arange(len(imglist)):
        dat = fits.getdata(directory+imglist[i], ext=0, header=False)
        ax[i].imshow(dat[int(py-1-rth):int(py-1+rth), int(px-1-rth):int(px-1+rth)],
                     origin="lower", cmap="gray_r", vmin=vmin, vmax=vmax)
        ax[i].tick_params(labelleft=False, labelbottom=False)
        
        ap_plt0 = RAp((px-1-int(px-1-rth), py-1-int(py-1-rth)),
                      w=ifu_w/pixel_scale, h=ifu_h/pixel_scale, theta=(gpa-wcs_angle)*np.pi/180.)
        ap_patches = ap_plt0.plot(axes=ax[i], color="red", lw=1.5, ls="-")
        
        if (ifu_h < 2*fwhm[i]):
            ap_plt = RAp((px-1-int(px-1-rth), py-1-int(py-1-rth)),
                         w=width[i], h=height[i], theta=(gpa-wcs_angle)*np.pi/180.)
            ap_patches = ap_plt.plot(axes=ax[i], color="green", lw=2.0, ls="-")

        if calc_sky:
            an_plt = RAn((px-1-int(px-1-rth), py-1-int(py-1-rth)),
                         w_in=w_in, w_out=w_out, h_in=h_in, h_out=h_out,
                         theta=(gpa-wcs_angle)*np.pi/180.)
            an_patches = an_plt.plot(axes=ax[i], color="magenta", lw=1.5, ls="--", alpha=0.7)

        ax[i].text(0.05, 0.95, filt[i], fontsize=20.0, fontweight="bold", color="black",
                   ha="left", va="top", transform=ax[i].transAxes)

        c0 = plt.Circle((0.15*dat[int(py-1-rth):int(py-1+rth), int(px-1-rth):int(px-1+rth)].shape[1],
                         0.30*dat[int(py-1-rth):int(py-1+rth), int(px-1-rth):int(px-1+rth)].shape[0]),
                        radius=fwhm[i]/pixel_scale/2, color='blue', fill=True, ls='-', lw=1.5, alpha=0.5)
        ax[i].add_patch(c0)

        if (i == 0):
            ax[i].text(0.05, 0.05, f"5 arcsec = {5*kpc_per_arcsec:.2f} kpc", fontsize=15.0, fontweight="bold", color="blueviolet",
                       ha="left", va="bottom", transform=ax[i].transAxes)
        
    plt.tight_layout()
    plt.savefig(out, dpi=300)        

plot_aperture("check_HST_ACS.png", diH, imglist, rth=150, vmin=-0.01, vmax=0.09)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# Aperture photometry for the IFU FOV
def run_photometry(directory, imglist):
    cols = [('id','U5'), ('x','<f4'), ('y','<f4'), 
            ('aperture_sum','<f4'), ('area_ap','<f4'),
            ('msky','<f8'), ('nsky','<f4'), ('sky_sigma','<f8'),
            ('source_sum','<f4'), ('mag','<f4'), ('merr','<f4')]
    phot_table = np.zeros(len(imglist), dtype=cols)

    for i in np.arange(len(imglist)):
        data, head = fits.getdata(directory+imglist[i], header=True, ext=0)
        zmag, gain, exptime, itime = head['MAGZERO'], head['CCDGAIN'], head['EXPTIME'], 1.0

        # Aperture photometry
        phot = apphot(data=data, apertures=ap[i])
        phot_table['id'][i] = filt[i]
        phot_table['x'][i], phot_table['y'][i] = phot['xcenter'].data[0], phot['ycenter'].data[0]
        phot_table['aperture_sum'][i] = phot['aperture_sum'].data[0]
        phot_table['area_ap'][i] = ap[i].area

        # Local sky estimation
        if calc_sky:
            sky_mask = an.to_mask(method='center')
            sky_vals = sky_mask.multiply(data)
            skyv = sky_vals[sky_vals != 0.]
            nsky = np.sum(sky_vals != 0.)
            avg, med, std = sigma_clipped_stats(skyv, sigma=2.5, maxiters=10, std_ddof=1)
            if med - avg < 0.3 * std:
                msky = med
            else:
                msky = 2.5 * med - 1.5 * avg

        # Magitude calculation
        src_sum = phot['aperture_sum'].data[0] - ap[i].area*msky
        nflx = src_sum / itime
        tflx = nflx * exptime
        mag = zmag - 2.5*np.log10(nflx) - Amag[i]
        err = np.sqrt(tflx/gain + ap[i].area*std**2. + (ap[i].area**2. * std**2.)/nsky)

        phot_table['msky'][i], phot_table['nsky'][i], phot_table['sky_sigma'][i] = msky, nsky, std
        phot_table['source_sum'][i], phot_table['mag'][i], phot_table['merr'][i] = src_sum, mag, (2.5/np.log(10.0)) * (err/tflx)
    
    return phot_table

phot_table = run_photometry(diH, imglist)
phot_table

array([('F435W', 196.83694, 179.06187,  98.59357, 14000., 0.0002111 , 15498., 0.00257525,  95.63816, 20.605963, 0.00156199),
       ('F606W', 196.83694, 179.06187, 354.26303, 14000., 0.00072152, 15498., 0.00375739, 344.16174, 20.084124, 0.00067724),
       ('F814W', 196.83694, 179.06187, 252.39742, 14000., 0.0005519 , 15498., 0.00286566, 244.67088, 19.92859 , 0.0006197 )],
      dtype=[('id', '<U5'), ('x', '<f4'), ('y', '<f4'), ('aperture_sum', '<f4'), ('area_ap', '<f4'), ('msky', '<f8'), ('nsky', '<f4'), ('sky_sigma', '<f8'), ('source_sum', '<f4'), ('mag', '<f4'), ('merr', '<f4')])

In [10]:
# Saving the results
m_AB_HST_ACS, e_m_AB_HST_ACS = phot_table['mag'], phot_table['merr']

### Aperture photometry for HST/WFC3-IR images

In [11]:
# Basic information
diH = "/data/jlee/DATA/HLA/McPartland+16/MACS1752/JFG2/Phot/"
imglist = ["110_ori.fits", "140_ori.fits"]
filt = ["F110W", "F140W"]
Amag = [0.027, 0.018]    # mag
fwhm = [0.2, 0.2]    # arcsec
pixel_scale = 0.05    # arcsec/pixel
wcs_angle = 0.029572035    # deg from DS9 box region

In [12]:
# WCS XY conversion for HST images
h = fits.getheader(diH+imglist[1], ext=0)
w = wcs.WCS(h)
px, py = w.wcs_world2pix(gra, gdec, 1)
print("# GMOS/IFU center")
print(f"(X, Y) = ({px:.4f}, {py:.4f})")

# GMOS/IFU center
(X, Y) = (197.8369, 180.0619)


In [13]:
# Creating the FOV aperture
calc_sky = True
w_in, w_out = ifu_w/pixel_scale+10, ifu_w/pixel_scale+60
h_in, h_out = ifu_h/pixel_scale+10, ifu_h/pixel_scale+60
ap, width, height, an = make_aperture(calc_sky=calc_sky, w_in=w_in, w_out=w_out, h_in=h_in, h_out=h_out)

In [14]:
# Showing the FOV aperture
plot_aperture("check_HST_WFC3_IR.png", diH, imglist, rth=150, vmin=-0.01, vmax=0.20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Aperture photometry for the IFU FOV
phot_table = run_photometry(diH, imglist)
phot_table

array([('F110W', 196.83694, 179.06187, 607.3199 , 14000., -0.00043788, 15498., 0.00503996, 613.45026, 19.825674, 0.00104352),
       ('F140W', 196.83694, 179.06187, 459.18646, 14000., -0.00128053, 15498., 0.00417098, 477.11383, 19.737762, 0.00118326)],
      dtype=[('id', '<U5'), ('x', '<f4'), ('y', '<f4'), ('aperture_sum', '<f4'), ('area_ap', '<f4'), ('msky', '<f8'), ('nsky', '<f4'), ('sky_sigma', '<f8'), ('source_sum', '<f4'), ('mag', '<f4'), ('merr', '<f4')])

In [16]:
# Saving the results
m_AB_HST_WFC3_IR, e_m_AB_HST_WFC3_IR = phot_table['mag'], phot_table['merr']

### Aperture photometry for GALEX images

In [17]:
# Basic information
diU = "/data/jlee/DATA/HLA/McPartland+16/MACS1752/test_SED/MAST/GALEX/"
imglist_int = ["fd-int.fits", "nd-int.fits"]
imglist_bgr = ["fd-skybg.fits", "nd-skybg.fits"]
imglist_ext = ["fd-rrhr.fits", "nd-rrhr.fits"]
filt = ["FUV", "NUV"]
E_bv = 0.03047207    # mag
cfac = [7.9, 8.0]    # Conversion factor (Gil de Paz+07)
Amag = np.array(cfac)*E_bv    # mag
# fwhm = [4.2, 5.3]    # arcsec
fwhm = [5.3, 5.3]    # arcsec (NUV)
pixel_scale = 1.5    # arcsec/pix
wcs_angle = 0.00020408708    # deg from DS9 box region

In [18]:
# WCS XY conversion for HST images
h = fits.getheader(diU+imglist_int[0], ext=0)
w = wcs.WCS(h)
px, py = w.wcs_world2pix(gra, gdec, 1)
print("# GMOS/IFU center")
print(f"(X, Y) = ({px:.4f}, {py:.4f})")

# GMOS/IFU center
(X, Y) = (3138.1476, 2543.3489)


In [19]:
# Creating the FOV aperture
calc_sky = False
ap, width, height = make_aperture(calc_sky=calc_sky)

In [20]:
# Showing the FOV aperture
plot_aperture("check_GALEX.png", diU, imglist_int, rth=10, vmin=0.0, vmax=0.02)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# Aperture photometry for the IFU FOV
cols = [('id','U5'), ('x','<f4'), ('y','<f4'), 
        ('aperture_sum','<f4'), ('area_ap','<f4'),
        ('msky','<f8'), ('nsky','<f4'), ('sky_sigma','<f8'),
        ('source_sum','<f4'), ('mag','<f4'), ('merr','<f4')]
phot_table = np.zeros(len(imglist_int), dtype=cols)

zeropoint = [18.82, 20.08]
for i in np.arange(len(imglist_int)):
    inten, hdr = fits.getdata(diU+imglist_int[i], header=True, ext=0)
    backgr = fits.getdata(diU+imglist_bgr[i], header=False, ext=0)
    eff_ext = fits.getdata(diU+imglist_ext[i], header=False, ext=0)

    mask_arr = np.zeros_like(inten).astype('bool')
    mask_arr[((inten == 0.) | (np.isnan(inten) == True))] = True

    # Aperture photometry
    phot = apphot(data=inten-backgr, apertures=ap[i],
                  error=np.sqrt(inten/eff_ext), mask=mask_arr)
    phot_table['id'][i] = filt[i]
    phot_table['x'][i], phot_table['y'][i] = phot['xcenter'].data[0], phot['ycenter'].data[0]
    phot_table['aperture_sum'][i] = phot['aperture_sum'].data[0]
    phot_table['source_sum'][i] = phot_table['aperture_sum'][i]
    phot_table['area_ap'][i] = ap[i].area

    mag = zeropoint[i] - 2.5*np.log10(phot['aperture_sum'].data[0]) - Amag[i]
    merr = (2.5/np.log(10.0)) * (phot['aperture_sum_err'].data[0]/phot['aperture_sum'].data[0])

    # Local sky estimation
    sky_mask = ap[i].to_mask(method='center')
    sky_vals = sky_mask.multiply(backgr)
    skyv = sky_vals[sky_vals != 0.]
    nsky = np.sum(sky_vals != 0.)
    avg, med, std = sigma_clipped_stats(skyv, sigma=2.5, maxiters=10, std_ddof=1)
    if med - avg < 0.3 * std:
        msky = med
    else:
        msky = 2.5 * med - 1.5 * avg

    phot_table['msky'][i], phot_table['nsky'][i], phot_table['sky_sigma'][i] = msky, nsky, std
    phot_table['mag'][i], phot_table['merr'][i] = mag, merr

phot_table

array([('FUV', 3137.1477, 2542.3489, 0.09503441, 69.91289, 0.00046259, 70., 2.21922226e-07, 0.09503441, 21.13457 , 0.23628151),
       ('NUV', 3137.1477, 2542.3489, 0.45420682, 69.91289, 0.00297897, 70., 8.60833916e-07, 0.45420682, 20.693089, 0.11980917)],
      dtype=[('id', '<U5'), ('x', '<f4'), ('y', '<f4'), ('aperture_sum', '<f4'), ('area_ap', '<f4'), ('msky', '<f8'), ('nsky', '<f4'), ('sky_sigma', '<f8'), ('source_sum', '<f4'), ('mag', '<f4'), ('merr', '<f4')])

In [22]:
# Saving the results
m_AB_GALEX, e_m_AB_GALEX = phot_table['mag'], phot_table['merr']

### Aperture photometry for Spitzer/IRAC images

In [23]:
# Basic information
diS = "/data/jlee/DATA/Spitzer/IRAC/MACS1752/r58320128/"
imglist_maic = sorted(glob.glob(diS+"*/pbcd/*_maic.fits"))
imglist_munc = sorted(glob.glob(diS+"*/pbcd/*_munc.fits"))
filt = [r"IRAC1 $(3.6\mu m)$", r"IRAC2 $(4.5\mu m)$"]
Amag = [0.0, 0.0]    # mag
fwhm = [1.95, 2.02]    # arcsec
pixel_scale = 0.6    # arcsec/pix
wcs_angle = 287.81478    # deg from DS9 box region

In [24]:
# WCS XY conversion for HST images
h = fits.getheader(imglist_maic[0], ext=0)
w = wcs.WCS(h)
px, py = w.wcs_world2pix(gra, gdec, 1)
print("# GMOS/IFU center")
print(f"(X, Y) = ({px:.4f}, {py:.4f})")

# GMOS/IFU center
(X, Y) = (1217.3134, 499.8611)


In [25]:
# Creating the FOV aperture
calc_sky = True
w_in, w_out = ifu_w/pixel_scale+3, ifu_w/pixel_scale+10
h_in, h_out = ifu_h/pixel_scale+3, ifu_h/pixel_scale+10
ap, width, height, an = make_aperture(calc_sky=calc_sky, w_in=w_in, w_out=w_out, h_in=h_in, h_out=h_out)

In [26]:
# Showing the FOV aperture
plot_aperture("check_Spitzer_IRAC.png", "", imglist_maic, rth=20, vmin=0.0, vmax=0.15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# Aperture photometry for the IFU FOV
cols = [('id','U5'), ('x','<f4'), ('y','<f4'), 
        ('aperture_sum','<f4'), ('area_ap','<f4'),
        ('msky','<f8'), ('nsky','<f4'), ('sky_sigma','<f8'),
        ('source_sum','<f4'), ('mag','<f4'), ('merr','<f4')]
phot_table = np.zeros(len(imglist_int), dtype=cols)

zeropoint = [18.80, 18.32]
flux0 = [280.9, 179.7]    # Jy
m0_AB = 23.93    # microJy
for i in np.arange(len(imglist_maic)):
    dat, hdr = fits.getdata(imglist_maic[i], header=True, ext=0)
    unc = fits.getdata(imglist_munc[i], header=False, ext=0)

    # Aperture photometry
    phot = apphot(data=dat, apertures=ap[i], error=unc)
    phot_table['id'][i] = filt[i]
    phot_table['x'][i], phot_table['y'][i] = phot['xcenter'].data[0], phot['ycenter'].data[0]
    phot_table['aperture_sum'][i] = phot['aperture_sum'].data[0]
    phot_table['area_ap'][i] = ap[i].area

    # Local sky estimation
    sky_mask = an.to_mask(method='center')
    sky_vals = sky_mask.multiply(dat)
    skyv = sky_vals[sky_vals != 0.]
    nsky = np.sum(sky_vals != 0.)
    avg, med, std = sigma_clipped_stats(skyv, sigma=2.5, maxiters=10, std_ddof=1)
    if med - avg < 0.3 * std:
        msky = med
    else:
        msky = 2.5 * med - 1.5 * avg

    # Magitude calculation
    src_sum = phot['aperture_sum'].data[0] - ap[i].area*msky
    mag = zeropoint[i] - 2.5*np.log10(src_sum) + (m0_AB - 2.5*np.log10(flux0[i]*1.0e+6)) - Amag[i]
    err = np.sqrt(phot['aperture_sum_err'].data[0]**2. + ap[i].area*std**2. + (ap[i].area**2. * std**2.)/nsky)
    merr = (2.5/np.log(10.0)) * (err/src_sum)

    phot_table['msky'][i], phot_table['nsky'][i], phot_table['sky_sigma'][i] = msky, nsky, std
    phot_table['source_sum'][i], phot_table['mag'][i], phot_table['merr'][i] = src_sum, mag, merr

phot_table

array([('IRAC1', 1216.3134, 498.86115, 5.583723, 97.22222, 0.02972232, 232., 0.00431977, 2.6940532, 20.532606, 0.03924139),
       ('IRAC2', 1216.3134, 498.86115, 9.738323, 97.22222, 0.07158738, 232., 0.0051473 , 2.778439 , 20.504128, 0.03600756)],
      dtype=[('id', '<U5'), ('x', '<f4'), ('y', '<f4'), ('aperture_sum', '<f4'), ('area_ap', '<f4'), ('msky', '<f8'), ('nsky', '<f4'), ('sky_sigma', '<f8'), ('source_sum', '<f4'), ('mag', '<f4'), ('merr', '<f4')])

In [28]:
# Saving the results
m_AB_Spitzer, e_m_AB_Spitzer = phot_table['mag'], phot_table['merr']

### Stacking the results & Comparing photometry

In [29]:
m_AB = np.hstack((m_AB_GALEX, m_AB_HST_ACS, m_AB_HST_WFC3_IR, m_AB_Spitzer))
e_m_AB = np.hstack((e_m_AB_GALEX, e_m_AB_HST_ACS, e_m_AB_HST_WFC3_IR, e_m_AB_Spitzer))
m_AB, e_m_AB

(array([21.13457 , 20.693089, 20.605963, 20.084124, 19.92859 , 19.825674,
        19.737762, 20.532606, 20.504128], dtype=float32),
 array([0.23628151, 0.11980917, 0.00156199, 0.00067724, 0.0006197 ,
        0.00104352, 0.00118326, 0.03924139, 0.03600756], dtype=float32))

In [30]:
# Old photometry
m_AB_0 = np.array([21.085, 20.626, 20.796, 20.2101, 20.0366, 19.9487, 19.8797, 20.593, 20.463])
e_m_AB_0 = np.array([0.23, 0.118, 0.1, 0.1, 0.1, 0.1, 0.1, 0.246, 0.197])

In [31]:
m_AB - m_AB_0

array([ 0.04956917,  0.06708853, -0.19003725, -0.12597639, -0.10800923,
       -0.12302594, -0.14193755, -0.06039388,  0.0411275 ])

In [32]:
hd = "HEADER\n"
hd += "(GALEX) FUV NUV\n"
hd += "(HST/ACS) F435W F606W F814W\n"
hd += "(HST/WFC3-IR) F110W F140W\n"
hd += "(Spitzer/IRAC) ch1[3.6um] ch2[4.5um]"
np.savetxt("phot_results.txt", np.column_stack([m_AB, e_m_AB]), fmt="%.4f  %.4f", header=hd)

When reading the result file,
```
mag, e_mag = np.loadtxt("phot_results.txt").T
```

### Estimation of stellar mass (w/ 3.6$\mu{\rm m}$ and 4.5$\mu{\rm m}$ flux)
* Refer to [Eskew et al. (2012)](https://iopscience.iop.org/article/10.1088/0004-6256/143/6/139/pdf)    
* Note that teh above literature uses the Salpeter IMF, so that the IMF calibration is needed if one uses different IMF.

In [33]:
def compute_Ms(m36, m45, lumdist, imf=0,
               error=True, e_m36=0.1, e_m45=0.1):
    '''
    m36: 3.6um AB magnitude
    m45: 4.5um AB magnitude
    imf: type of the initial mass function (IMF)
        0 - Salpeter 1955 IMF (default)
        1 - Kroupa 2001 IMF
        2 - Chabrier 2003 IMF
    error: error propagation or not (default: True)
    e_m36: uncertainty of 3.6um AB magnitude
    e_m45: uncertainty of 4.5um AB magnitude
    '''
    f_v36 = 10.0**(-0.4*(m36 - 8.90))   # AB mag --> Jy 
    f_v45 = 10.0**(-0.4*(m45 - 8.90))   # AB mag --> Jy
    Ms_0 = 10.0**5.65 * f_v36**2.85 * f_v45**(-1.85) * (lumdist/0.05)**2.0
    
    # Salpeter IMF
    if (imf == 0):
        Ms = Ms_0
    
    # Kroupa IMF
    if (imf == 1):
        Ms = Ms_0 * 0.626
    
    # Chabrier IMF
    if (imf == 2):
        Ms = Ms_0 * 0.542
    
    # Error propagation
    if error:
        e_f_v36 = 0.4*np.log(10)*e_m36*f_v36
        e_f_v45 = 0.4*np.log(10)*e_m45*f_v45
        e_Ms_0 = 10.0**5.65 * (lumdist/0.05)**2.0
        e_Ms_0 *= np.sqrt((2.85 * f_v36**1.85 * f_v45**(-1.85) * e_f_v36)**2.0 + \
                          (1.85 * f_v36**2.85 * f_v45**(-2.85) * e_f_v45)**2.0)
        
        if (imf == 0):
            e_Ms = e_Ms_0
        if (imf == 1):
            e_Ms = e_Ms_0 * 0.626
        if (imf == 2):
            e_Ms = e_Ms_0 * 0.542
        
        return [Ms, e_Ms]
    
    else:
        return [Ms, None]

In [34]:
Ms, e_Ms = compute_Ms(m_AB_Spitzer[0], m_AB_Spitzer[1], ldist, imf=2,
                      error=True, e_m36=0.1, e_m45=0.1)
print("Stellar mass from NIR fluxes")
print(f"Ms = {Ms:.3e} Mo +/- {e_Ms:.3e} Mo")
print(f"log Ms/Mo = {np.log10(Ms):.3f} +/- {e_Ms/(Ms*np.log(10.0)):.3f}")

Stellar mass from NIR fluxes
Ms = 7.200e+09 Mo +/- 2.253e+09 Mo
log Ms/Mo = 9.857 +/- 0.136
